In [40]:
import spiceypy as spice
import pvl
import os
import re
import subprocess
import ale
import io
import re
import networkx as nx

# $messenger = /usgs/cpkgs/isis3/data/messsenger/
# $base = /usgs/cpkgs/isis3/data/base/

# Grabs the kernels and appends them to independent lists
def check_kern(kern):
    kern = kern.replace("$messenger/", os.environ['messenger'])
    kern = kern.replace("$base/", os.environ['base'])    
    filename, file_extension = os.path.splitext(kern)
    if(file_extension == '.bc'):
        bc_kerns.append(kern)
    elif(file_extension == '.tls'):
        tls_kerns.append(kern)
    elif(file_extension == '.tf'):
        tf_kerns.append(kern)
    elif(file_extension == '.tsc'):
        tsc_kerns.append(kern)
    elif(file_extension == '.bsp'):
        spk_kerns.append(kern)
    elif(file_extension == '.ti'):
        ti_kerns.append(kern)
    elif(file_extension == '.tpc'):
        tpc_kerns.append(kern)
    else:
        print("'{}' is not a valid kernel".format(kern))
        return 0
    
def get_intervals(bc_filename):
    interval = subprocess.check_output(["ckbrief", bc_filename])
    intervals = []

    for line in interval.splitlines():
        line = str(line)
        if('Object:' in line):
            intervals.append(int(line.strip("''").split()[1]))
            
    return intervals

def light_travel(kernels, sclk_start, sclk_stop, UTC=False):
    merc_id = 199
    mess_id = -236
    
    furnished_kerns = spice.furnsh(kernels)
    
    mess_state, mess_lt = spice.spkez(mess_id, (sclk_start + sclk_stop)/2, 'J2000', 'LT+S', 0)
    merc_state, merc_lt = spice.spkez(merc_id, (sclk_start + sclk_stop)/2, 'J2000', 'LT+S', 0)
    
    lt_pad = max(abs(mess_lt), abs(merc_lt)) + 15
    
    # Eph Time
    padded_start_et = sclk_start - lt_pad
    padded_end_et = sclk_stop + lt_pad
    
    # SCLK Padded Time
    padded_start_sclk = spice.sce2s(mess_id, padded_start_et)
    padded_end_sclk = spice.sce2s(mess_id, padded_end_et)
    
    # UTC Padded Time 
    if(UTC):
        padded_start_utc = spice.et2utc(padded_start_et, 'c', 3)
        padded_end_utc = spice.et2utc(padded_end_et, 'c', 3)
        
        return padded_start_utc, padded_end_utc
    
    return padded_start_sclk, padded_end_sclk
    
def ckslicer(bc_kern, tsc_kern, tls_kern, sclk_start, sclk_stop, reference_id):

    ck_filename, ck_file_extension = os.path.splitext(bc_kerns[0]) 
    #     sclk_start, sclk_stop =scl 
    sclk_stop = spice.sce2s(spacecraft_id, spice.scs2e(spacecraft_id, sclk_start) + exp_dur)

    ckslicer_command = ["./ckslicer", 
                        '-LSK {}'.format(tls_kern), 
                        '-SCLK {}'.format(tsc_kern), 
                        '-INPUTCK {}'.format(ck_filename + ck_file_extension), 
                        '-OUTPUTCK {}'.format("~/Desktop/ck" +'_sliced' + ck_file_extension),
                        '-ID {}'.format(reference_id),
                        '-TIMETYPE {}'.format('SCLK'),
                        '-START {}'.format(sclk_start),
                        '-STOP {}'.format(sclk_stop)]
    
    return subprocess.call(ckslicer_command), ckslicer_command  

In [2]:
# !ckbrief

# Enviorment variable that will be used to grab isis3 data
os.environ['messenger'] = "/usgs/cpkgs/isis3/data/messenger/"
os.environ['base'] = "/usgs/cpkgs/isis3/data/base/"
os.environ['spk_home'] = "/usgs/cpkgs/isis3/data/messenger/kernels/tspk/"

# Cube image path
cube_lab = "/work/users/sgstapleton/kernel_split/EN1072174528M.cub"

# Loads cube to extract data
cube_pvl = pvl.load(cube_lab)

# The kernels that the cub uses
parsed_kerns = cube_pvl['IsisCube']['Kernels']

# Starting SCLK string and ExposureDuration
sclk_start = cube_pvl['IsisCube']['Instrument']['SpacecraftClockCount']
exp_dur = cube_pvl['IsisCube']['Instrument']['ExposureDuration'].value * 0.001

# The fields we care about in the cub
required_elems = ['LeapSecond', 'TargetAttitudeShape', 'TargetPosition', 'InstrumentPointing', 'Instrument', 
                 'SpacecraftClock', 'InstrumentPosition', 'InstrumentAddendum']

# Lists to be used to store kernel vars
kernel_list = []
bc_kerns = []
tls_kerns = []
tf_kerns = []
tsc_kerns = []
spk_kerns = []
ti_kerns = []
tpc_kerns = []

for kern in required_elems:
    kern = parsed_kerns[kern]
    if(isinstance(kern, list)):
        for kernel in kern:
            check_kern(kernel)
    else:
        check_kern(kern)

space_intervals = get_intervals(bc_kerns[0])
target_spk_file = os.path.join(os.environ['spk_home'], "de423s.bsp")
kernels = bc_kerns + tsc_kerns + tls_kerns + spk_kerns + ti_kerns + tf_kerns + tsc_kerns

# Spacecraft / Reference ID of CK kerns
spacecraft_id = int(str(space_intervals[0])[:-3])
reference_id = int(str(space_intervals[0]))

# Gets SCLK times
spice.furnsh(kernels)
sclk_start = spice.scs2e(spacecraft_id, sclk_start)
sclk_stop = sclk_start + exp_dur

# Gets light corrected SCLK times
padded_sclk_start, padded_sclk_stop = light_travel(kernels, sclk_start, sclk_stop)

call, cmd = ckslicer(bc_kerns[0], tsc_kerns[0], tls_kerns[0], padded_sclk_start, padded_sclk_stop, reference_id)

'Table' is not a valid kernel
'Table' is not a valid kernel
'Table' is not a valid kernel


In [3]:
# Remove comments from the ck file
# NOTE: '~' will not work, use full path
new_kern = '/Users/tthatcher/Desktop/ck' +'_sliced' + '.bc'
commnt_command = ['commnt', '-d {}'.format(str(new_kern))]
subprocess.call(commnt_command)

# Makes a new txt file for the only comments that should be stored in the new ck file
with open("temp_commnts.txt","w+") as f:
    first_line = "This CK is for testing with the image: {}".format(cube_lab)
    second_line = "\nThis CK was generated using the following command: {}".format(" ".join(cmd))
    f.write(first_line + second_line)

new_commnts_command = ["commnt", "-a {}".format(new_kern), "temp_commnts.txt"]
subprocess.call(new_commnts_command)

1

In [4]:
# Transfer the subsetted ck into a xfr file
toxfr_command = "toxfr", "{}".format(new_kern)
subprocess.call(toxfr_command)

1

In [33]:
baseSPK = '''         SOURCE_SPK_KERNEL   = <the SPK your are extracting from>
            INCLUDE_COMMENTS = no
            BODIES              = <the ID code of either your spacecraft or target>
            BEGIN_TIME          = <the start time of your image as a UTC string>
            END_TIME            = <the stop time of your image as a UTC stirng>'''.format(tls_kerns[0])


f = io.StringIO(baseSPK)
line = f.readline()
spk_original = '/work/users/kernel_split/{}'.format(spk_kerns[0])
filename, file_extension = os.path.splitext(spk_original)
spk_created = filename + "_merged" + file_extension
with open("conf.txt", "w+") as real_file:
    real_file.write('''LEAPSECONDS_KERNEL     = {}\n'''.format(tls_kerns[0]))
    real_file.write('''      SPK_KERNEL             = {}\n'''.format(spk_created))
    
    for 
    
    while(line):
    #     f.write(line.replace("<Your LSK kernel>", tls_kerns[0]))
        real_file.write(line)
#         print(line)
        line = f.readline()
    real_file.close()


f.seek(0)
f.readlines()
x = open("conf.txt", "r")
x.readlines()
# print(spk_kerns)

['LEAPSECONDS_KERNEL     = /usgs/cpkgs/isis3/data/base/kernels/lsk/naif0012.tls\n',
 '      SPK_KERNEL             = /work/users/kernel_split//usgs/cpkgs/isis3/data/messenger/kernels/tspk/de423s_merged.bsp\n',
 '         SOURCE_SPK_KERNEL   = <the SPK your are extracting from>\n',
 '            INCLUDE_COMMENTS = no\n',
 '            BODIES              = <the ID code of either your spacecraft or target>\n',
 '            BEGIN_TIME          = <the start time of your image as a UTC string>\n',
 '            END_TIME            = <the stop time of your image as a UTC stirng>']

In [6]:
# !commnt -d ~/Desktop/ck_sliced.bc
# !./ckslicer -LSK /usgs/cpkgs/isis3/data/base/kernels/lsk/naif0012.tls -SCLK /usgs/cpkgs/isis3/data/messenger/kernels/sclk/messenger_2548.tsc -INPUTCK /usgs/cpkgs/isis3/data/messenger/kernels/ck/msgr_1504_v01.bc -OUTPUTCK ~/Desktop/ck_sliced.bc -ID -236001 -TIMETYPE SCLK -START "2/0072174528:989000" -STOP "2/072174528:990000"

In [7]:
print(cube_pvl)

PVLModule([
  ('IsisCube',
   {'Archive': {'CCDTemperature': 1139,
                'DataQualityId': 1000000000,
                'DataSetId': 'MESS-E/V/H-MDIS-2-EDR-RAWDATA-V1.0',
                'EdrProductCreationTime': datetime.datetime(2015, 4, 30, 18, 25, 23),
                'EdrSourceProductId': '1072174528_IM6',
                'Exposure': 1,
                'MissionElapsedTime': 72174528,
                'ObservationId': 8386282,
                'ObservationStartTime': datetime.datetime(2015, 4, 24, 4, 42, 19, 666463),
                'ObservationType': ['Monochrome', 'Ridealong NAC'],
                'OrbitNumber': 4086,
                'OriginalFilterNumber': 0,
                'ProducerId': 'APPLIED COHERENT TECHNOLOGY CORPORATION',
                'ProductId': 'EN1072174528M',
                'SequenceName': 'N/A',
                'SiteId': 'N/A',
                'SourceProductId': ['EN1072174528M', 'MDISLUTINV_0'],
                'SpacecraftClockStartCount': '2/0072174528

In [8]:
spaccraft_name = cube_pvl['IsisCube']['Instrument']['SpacecraftName']
target_name = cube_pvl['IsisCube']['Instrument']['TargetName']

# Convert strings to int ID's
craft_id = spice.bods2c(spaccraft_name)
target_id = spice.bods2c(target_name)
print(target_id)

199


In [79]:
                
def get_body_ids(spk_file, body_id):
    brief = subprocess.check_output(["brief", "-c {}".format(spk_file)])
    
    # Convert from bytes
    brief = brief.decode("utf-8")
    print(brief)
    brief_io = io.StringIO(brief)
    line = brief_io.readline()
    edge_list = []
        
    while(line):
        
        if("w.r.t" in line):
                bodies_results = re.findall('\((.*?)\)', line)
                edge_list.append(bodies_results)
                        
        line = brief_io.readline()
        
    G = nx.DiGraph()
    G.add_edges_from(edge_list)
    
    body_ids = []
    for path in nx.all_simple_paths(G, body_id, '0'):
        body_ids = body_ids + path
        
    body_ids = set(body_ids)

    return body_ids

file = "/usgs/cpkgs/isis3/data/messenger/kernels/spk/msgr_20040803_20150430_od431sc_2.bsp"
# ids = get_spk_ids(file)
# print(ids)
# x = search_string(file)
# print(x)
# giant_one = []
# for path in nx.all_simple_paths(x, '-236', '0'):
#     giant_one = giant_one + path
    
# not_so_giant_one = set(giant_one)
# print(not_so_giant_one)

In [81]:

def output_config(spk_kerns, tls_kerns):

    spk_kerns_merged = []
    
    for kern in spk_kerns:
        
        filename, file_extension = os.path.splitext(kern)
        kern_merged_basename = os.path.basename(kern)
        kern_merged = kern_merged_basename + "_merged" + file_extension
        kern_merged_config = kern_merged_basename + "_merged.config"

        with open(kern_merged_config, "w+") as config:
            config.write('''LEAPSECONDS_KERNEL     = {}\n'''.format(tls_kerns[0]))
            config.write('''      SPK_KERNEL             = {}\n'''.format(kern_merged))
            
            body_ids = get_body_ids(kern, '1')

            for body in body_ids:

                if body != '0':

                    config.write('''         SOURCE_SPK_KERNEL   = {}\n'''.format(kern_merged))
                    config.write('''            INCLUDE_COMMENTS = no\n''')
                    config.write('''            BODIES              = {}\n'''.format(body))
                    config.write('''            BEGIN_TIME          = {}\n'''.format("hi"))
                    config.write('''            END_TIME            = {}\n'''.format("hi"))
    


# f.seek(0)
# f.readlines()
# x = open("conf.txt", "r")
# x.readlines()

output_config(spk_kerns, tls_kerns)


 
BRIEF -- Version 4.0.0, September 8, 2010 -- Toolkit Version N0066
 
 
Summary for: /usgs/cpkgs/isis3/data/messenger/kernels/tspk/de423s.bsp
 
Bodies: MERCURY BARYCENTER (1) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        VENUS BARYCENTER (2) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        EARTH BARYCENTER (3) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        MARS BARYCENTER (4) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        JUPITER BARYCENTER (5) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        SATURN BARYCENTER (6) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        URANUS BARYCENTER (7) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        NEPTUNE BARYCENTER (8) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        PLUTO BARYCENTER (9) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        SUN (10) w.r.t. SOLAR SYSTEM BARYCENTER (0)
        MERCURY (199) w.r.t. MERCURY BARYCENTER (1)
        VENUS (299) w.r.t. VENUS BARYCENTER (2)
        MOON (301) w.r.t. EARTH BARYCENTER (3)
        EARTH (399) w.r.t. EARTH BARYCENTER (3)
        MARS (499) w.r.t.